In [25]:
result_path = "../runs/compare_GraphOps_2"

train_only = False

import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.python.summary.summary_iterator import summary_iterator
import numpy as np
from scipy import stats

from sklearn.metrics import r2_score

In [26]:
def equalize(trains, vals):
    
    
    if (len(vals) == 0):
        vals = np.zeros(len(trains))
    else:
        vals=np.repeat(vals,10)
        vals = vals[:-(len(vals)-len(trains))]
    df = pd.DataFrame({"train":trains, "val":vals})
    df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]
    
    return df

for res in os.listdir(result_path):
    
    print(res)
    
    events = None
    for f in os.listdir(os.path.join(result_path, res)):
        if f.startswith("events.out"):
            events = f
            
    trains = []
    vals = []

    edges_trains = []
    edges_vals = []

    node_trains = []
    node_vals = []

    node_accuracy = []
    node_accurcy_val = []
    
    if events is None:
        continue

    for e in summary_iterator(os.path.join(result_path, res, events)):
        for v in e.summary.value:
            if v.tag == 'Loss/train':
                trains.append(v.simple_value)
            if v.tag == 'Loss/val':
                vals.append(v.simple_value)
            if v.tag == 'Loss_edges/train':
                edges_trains.append(v.simple_value)
            if v.tag == 'Loss_edges/val':
                edges_vals.append(v.simple_value)
            if v.tag == 'Loss_nodes/train':
                node_trains.append(v.simple_value)
            if v.tag == 'Loss_nodes/val':
                node_vals.append(v.simple_value)
            if v.tag == 'Node_Accuracy/train':
                node_accuracy.append(v.simple_value)
            if v.tag == 'Node_Accuracy/val':
                node_accurcy_val.append(v.simple_value)

    results = {}
    if len(trains)>0:
        loss= equalize(trains, vals)
        results["Total Loss"]=loss
    if len(edges_trains)>0:
        edge_loss = equalize(edges_trains,edges_vals)
        results["Edge reconstruction loss"]=edge_loss
    if len(node_trains)>0:
        node_loss = equalize(node_trains,node_vals)
        results["Node classification loss"]=node_loss

    if len(node_accuracy)>0:
        node_accuracy = equalize(node_accuracy,node_accurcy_val)
        results["Node classification training-accuracy"]=node_accuracy

    fig, axs = plt.subplots(2,2, figsize=(15, 15), facecolor='w', edgecolor='k')
    axes = axs.ravel()
    for i, df_key in enumerate(results.keys()):

        df = results[df_key]
        ax = axes[i]

        ax.plot(df["train"], color='blue')
        if not train_only:
            
            ax.plot(df['val'], color='orange')
            ax.legend(labels=['training','validation'])
        else:
            ax.legend(labels=['training'])

        ax.set_title(df_key)
        
    if not os.path.exists(os.path.join(result_path, res, "Figures")):
        os.makedirs(os.path.join(result_path, res, "Figures"))
    plt.savefig(os.path.join(result_path, res, "Figures","Training_Losses"))
    
    if os.path.exists(os.path.join(result_path, res, "test", "pred_test.csv")):
        p = pd.read_csv(os.path.join(result_path, res, "test", "pred_test.csv"))

        fig, ax = plt.subplots()

        ax.scatter(p["pred_mrl"], p["true_mrl"], alpha=0.3, edgecolors='none')
        ax.text(3, 8, "R2: {:.3f}".format(r2_score(p["pred_mrl"], p["true_mrl"])))
        ax.set_title("Performance on test set")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("True")
        plt.savefig(os.path.join(result_path, res, "Figures","Test-set Loss"))

CNN


DataLossError: truncated record at 4154152' failed with Read less bytes than requested